https://huggingface.co/datasets/Qdrant/NOAA-Buoy

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [17]:
data = pd.read_csv('./full_years_remove_flawed_rows.csv')
data.head()

,TSTMP,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP
0,1980-01-01 00:00:00-05:00,1980,1,1,0,0,323.0,10.6,14.3,1.8,5.9,5.1,NaN,1020.2,15.6,21.1
1,1980-01-01 01:00:00-05:00,1980,1,1,1,0,317.0,10.0,13.0,1.7,6.3,5.0,NaN,1020.7,15.6,21.1
2,1980-01-01 02:00:00-05:00,1980,1,1,2,0,323.0,9.4,12.3,1.8,6.3,4.9,NaN,1021.3,15.6,21.1
3,1980-01-01 03:00:00-05:00,1980,1,1,3,0,311.0,9.3,12.3,1.8,6.3,5.1,NaN,1021.8,15.7,21.2
4,1980-01-01 04:00:00-05:00,1980,1,1,4,0,324.0,8.2,10.9,1.8,6.7,5.2,NaN,1022.0,15.4,21.2


In [18]:
data = data.drop('#YY',axis=1)
data = data.drop('mm', axis=1)
data = data.drop('hh', axis=1)
data.head()

,TSTMP,MM,DD,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP
0,1980-01-01 00:00:00-05:00,1,1,323.0,10.6,14.3,1.8,5.9,5.1,NaN,1020.2,15.6,21.1
1,1980-01-01 01:00:00-05:00,1,1,317.0,10.0,13.0,1.7,6.3,5.0,NaN,1020.7,15.6,21.1
2,1980-01-01 02:00:00-05:00,1,1,323.0,9.4,12.3,1.8,6.3,4.9,NaN,1021.3,15.6,21.1
3,1980-01-01 03:00:00-05:00,1,1,311.0,9.3,12.3,1.8,6.3,5.1,NaN,1021.8,15.7,21.2
4,1980-01-01 04:00:00-05:00,1,1,324.0,8.2,10.9,1.8,6.7,5.2,NaN,1022.0,15.4,21.2


In [19]:
data['TSTMP'] = pd.to_datetime(data['TSTMP'])
data['TSTMP']

0        1980-01-01 00:00:00-05:00
1        1980-01-01 01:00:00-05:00
2        1980-01-01 02:00:00-05:00
3        1980-01-01 03:00:00-05:00
4        1980-01-01 04:00:00-05:00
                    ...           
332886   2022-12-31 21:40:00-05:00
332887   2022-12-31 22:10:00-05:00
332888   2022-12-31 22:40:00-05:00
332889   2022-12-31 23:10:00-05:00
332890   2022-12-31 23:40:00-05:00
Name: TSTMP, Length: 332891, dtype: datetime64[ns, UTC-05:00]